In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import codecs
import pickle
import pandas as pd
import re
from sklearn.metrics.pairwise import linear_kernel
from konlpy.tag import Okt
import numpy as np

In [2]:
df = pd.read_excel('식당_리뷰추가_2024-08-29.xlsx')
df.head()

,검색어,title,category,양식,브런치,일식,일식당,이탈리아음식,스파게티,파스타전문,...,초밥뷔페,두부요리,치킨,닭강정,순대,순댓국,복어요리,닭요리,프랑스음식,review
0,성수카페거리,37.5 시그니처 성수,양식>브런치,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"두 가지 다 너무 맛있어요!브런치는 잼, 휘핑, 시럽 곁들여 먹을 소스가 많아서 좋..."
1,성수카페거리,탐광,음식점>일식>일식당,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,부추가 정말정말 맛있었던 탐광입니다자세한 이야기는 차차 하도록 하구요일단은 키오스크...
2,성수카페거리,타이거풀 성수,음식점>양식,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"친구 대학원 졸업 기념으로 방문했는데, 식당 분위기가 너무 좋고 음식이 맛있어서 좋..."
3,성수카페거리,성수다락,"이탈리아음식>스파게티,파스타전문",0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,NaN
4,성수카페거리,bd버거 성수,음식점>햄버거,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,풍자씨가 성수동에서 또간집 추천할 곳 알려 달라고 하면 보다버거 이야기할 거예요. ...


In [3]:
print(df.isnull().sum())

검색어          0
title        0
category     0
양식           0
브런치          0
            ..
순댓국          0
복어요리         0
닭요리          0
프랑스음식        0
review      87
Length: 73, dtype: int64


In [4]:
df.fillna('', inplace=True)

In [5]:
okt = Okt()

In [6]:
stop_words = '곳 것 는 두 다 가 입니다 으로 했는데 너무 에 이나 할 거 더 층 때 와 방문 등 저 이 현대 정말 길 바로 하나 안 그 날 주차 구경 번 볼 중 자리 위치 느낌 수 보다 하다 이다 있다 이용 분 집 점 정도 건물 보고 테이블 주차장 여기 요 전시 제 소개 앞 우리 나 또 전 내 위 꼭 후 다른 듯 진행 매장 좀 조금 마을 처음 만 다시 진짜 가지 때문 향 게 쪽 알 가장 요즘 지금 지하 도 속 출구 옆 한번 역시 참고 모두 온 말 및 구석구석 그냥 위해 꽤 아래 걸 로 먼저 통해 저희 직접 이상 사실 가능 시 살짝 마지막 여러 함 뒤 세 호선 명 기본 미리 제일 약 뭐 건 경우 어디 를 의 사이 편 가기 가면 동안 잠시 기간 루프 토 확인 끝 중간 거의 일단 기준 마치 이후 자주 외 모든 첫 의자  만큼 데 왜 그대로 하니 원래 동시 총 월 부분 방법 뭔가 순간미니 별로 평소 또한 문래 채 월요일 마무리 매우 방 진 밖 은 몇 뿐 골 앱 지난 일요일 리 늘 박 상 랍니 워낙 화 선 무엇 지나 컷 겸 주 비롯 여 자신 과 고 네 너 니 원'
stop_words = stop_words.split()
stop_words[:15]

['곳',
 '것',
 '는',
 '두',
 '다',
 '가',
 '입니다',
 '으로',
 '했는데',
 '너무',
 '에',
 '이나',
 '할',
 '거',
 '더']

In [7]:
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    tokens = okt.morphs(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['review_after'] = df['review'].apply(preprocess_text)
df.head()

,검색어,title,category,양식,브런치,일식,일식당,이탈리아음식,스파게티,파스타전문,...,두부요리,치킨,닭강정,순대,순댓국,복어요리,닭요리,프랑스음식,review,review_after
0,성수카페거리,37.5 시그니처 성수,양식>브런치,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"두 가지 다 너무 맛있어요!브런치는 잼, 휘핑, 시럽 곁들여 먹을 소스가 많아서 좋...",맛있어요 브런치 잼 휘핑 시럽 곁들여 먹을 소스 많아서 좋았고 있는 빵 소시지 따로...
1,성수카페거리,탐광,음식점>일식>일식당,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,부추가 정말정말 맛있었던 탐광입니다자세한 이야기는 차차 하도록 하구요일단은 키오스크...,부추 맛있었던 탐광 자세한 이야기 차차 하도록 하구요 키 오스 크에서 결제 하고 안...
2,성수카페거리,타이거풀 성수,음식점>양식,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"친구 대학원 졸업 기념으로 방문했는데, 식당 분위기가 너무 좋고 음식이 맛있어서 좋...",친구 대학원 졸업 기념 식당 분위기 좋고 음식 맛있어서 좋았어용 봉골레 스튜 파스타...
3,성수카페거리,성수다락,"이탈리아음식>스파게티,파스타전문",0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,,
4,성수카페거리,bd버거 성수,음식점>햄버거,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,풍자씨가 성수동에서 또간집 추천할 곳 알려 달라고 하면 보다버거 이야기할 거예요. ...,풍자 씨 성수동 에서 간 추천 알려 달라 하면 버거 이야기 예요 와사비 슈 림프 최...


In [8]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['review_after'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [50]:
def content_based_filtering(search_term, category_term, review_term, df, cosine_sim, top_n=5):
    # 리뷰 칼럼에서 빈 칸 제외
    df = df[df['review'].str.strip() != '']
    
    # 검색어로 필터링
    search_matches = df[df['검색어'].str.contains(search_term, na=False)]
    
    if search_matches.empty:
        return "검색어에 해당하는 결과가 없습니다."
    
    # Title 값이 같은 항목 그룹화
    grouped_by_title = search_matches.groupby('title')
    
    results = []
    
    for title, group in grouped_by_title:
        # 카테고리 필터링 (최우선)
        category_filtered = group[group['category'].str.contains(category_term, na=False)]
        
        if not category_filtered.empty:
            results.append(category_filtered)
    
    if results:
        final_results = pd.concat(results).drop_duplicates(subset='review')
        
        if len(final_results) >= top_n:
            return final_results.head(top_n)
    
    # 카테고리 필터링된 결과가 부족할 경우, 리뷰로 추가 필터링
    for title, group in grouped_by_title:
        review_filtered = group[group['review'].str.contains(review_term, na=False)]
        
        if not review_filtered.empty:
            results.append(review_filtered)
    
    if results:
        final_results = pd.concat(results).drop_duplicates(subset='review')
        
        if len(final_results) >= top_n:
            return final_results.head(top_n)
    
    # 리뷰 필터링 결과가 부족할 경우, 빈 리뷰가 아닌 항목으로 랜덤 채우기
    all_titles = search_matches['title'].unique()
    for title in all_titles:
        title_matches = search_matches[search_matches['title'] == title]
        non_empty_review = title_matches[title_matches['review'].str.strip() != '']
        
        if not non_empty_review.empty:
            results.append(non_empty_review)
    
    final_results = pd.concat(results).drop_duplicates(subset='review')
    
    if len(final_results) >= top_n:
        return final_results.head(top_n)
    
    # 최종적으로 랜덤하게 결과 채우기
    if not search_matches.empty:
        return search_matches.sample(n=top_n, replace=True).drop_duplicates(subset='review')
    
    return search_matches.sample(n=1)
    
# 사용자로부터 검색어, 카테고리, 리뷰 검색어 입력받기
search_term = input("검색어를 입력하세요: ")
category_term = input("카테고리에서 검색할 단어를 입력하세요: ")
review_term = input("리뷰에서 검색할 단어를 입력하세요: ")

# 추천 항목 가져오기
recommended_items = content_based_filtering(search_term, category_term, review_term, df, cosine_sim)

print(recommended_items[['검색어', 'title', 'category', 'review']])

검색어를 입력하세요:  대공원
카테고리에서 검색할 단어를 입력하세요:  양식
리뷰에서 검색할 단어를 입력하세요:  맛


       검색어          title    category  \
237  서울대공원           윕 성수      음식점>양식   
230  서울대공원      을지다락 도산공원      음식점>양식   
234  서울대공원   고베규카츠 서울 강남점  음식점>일식>돈가스   
233  서울대공원         온센 강남점  음식점>일식>일식당   
232  서울대공원  이자카야 공감 가로수길점     술집>이자카야   

                                                review  
237  성수에 일이있어 와이프 데리고 방문한 윕 성수! 초당옥수수를 좋아하다보니 뇨끼에 끌...  
230  오므라이스가 시그니처메뉴답게 인기가 가장 많네요 오므라이스를 안시킨 테이블이 없어요...  
234  다섯명이서 가서 자리가 잘 안날줄 알았는데 생각보다 회전율이 빨라서 거의 바로 앉았...  
233  오픈 시간 맞춰 갔는데도 기다리고 있는 사람이 꽤 있었어요. 맛있고 친절하고 가격도...  
232  데이트겸해서 방문했던곳인데 안주들이 맛있어서 좋았어요👍🏻 다른 메뉴들도 맛있았는데 ...  


In [53]:
!pip install streamlit-modal